In [ ]:
%pip install numpy
%pip install sympy
%pip install scipy
%load_ext autoreload
%autoreload 2
from simplex_two_phases import simplex as simplex_two_phases, _simplex_find_feasible_initial_basis
from simplex_tableau import simplex as simplex_tableau
#from simplex_tableau import markdown_repr_T
from simplex import simplex
from revised_simplex_tableau import revised_simplex_tableau, markdown_repr_T
from utils import array_to_markdown

    
import numpy as np
from pprint import pprint
from IPython.display import Markdown, display
from itertools import combinations
from sympy import symbols, Matrix, solve
from scipy.optimize import linprog


solution_types = {
    -1: 'Unfeasible',
    1: 'Optimal finite solution found',
    2: 'Multiple optimal solutions found',
    3: 'Unbounded'
}

# Exercicio 4.1

Verificando se as restrições de não negatividade para $ {A_{I}}^{-1} $ são violadas

In [ ]:
A = np.array([
    [ 1, 1, -1, 0, 0],
    [-2, 3,  0, 1, 0],
    [ 0, 1,  0, 0, 1]
])

B = np.array([
    1,
    6,
    2
])

C = np.array([1, -3, 0, 0, 0])

I = [2, 3, 4]

A_I = A[:,I] # definição da matriz básica
A_I_inv = np.linalg.inv(A_I)

result = f'Restrições de negatividade{
    ' não ' if np.all(A_I_inv @ B >= 0) else ' '
}violadas'

display(
    Markdown(
        result
    )
)

Como as restrições de negatividade foram violadas, devemos criar um problema artificial e resolvê-lo para:

1. Verificar a factibilidade do problema original
2. Encontrar uma base inicial factível para o problema original que seja capaz de resolvê-lo

O problema artificial é do tipo:

$$
\begin{aligned}
    & \text{min} \quad \phi = & \textbf{1} \: x_a \\
    & \text{s. t.} & A x + x_a &= b \\
    &              &       x   &\geq 0 \\
    &              &       x_a &\geq 0 \\
\end{aligned}
$$

Vamos resolvê-lo abaixo:

In [ ]:

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = [5, 3, 4]

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)

Verificamos que $ \phi^{*} = 0 $, o que caracteriza o problema original como factível.

Portanto, vamos resolvê-lo à partir da base inicial facítvel obtida na resolução do artificial $ I = [0, 3, 4] $

In [ ]:
I = [0, 3, 4]

# resolvendo problema original com o simplex tableau revisado
solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)

## Result Comparison through scipy

In [ ]:
result = linprog(C, A_eq = A, b_eq = B, method='highs')
print('Result comparison, we found the optimal solution through our tableaus')
display(result)

## Result Comparison through Simplex Two Phases (without tableaus)

In [ ]:
I = [2, 3, 4]
_, n = A.shape
X = np.zeros(n)
z_star, x_star, I_star, A_I_star, A, iterations_count, solution_type, debug_info  = simplex_two_phases(A, B, C, I, debug=True)
X[I_star] = x_star

result_md = f'''
Solution Type: {solution_types[solution_type]}

$ Z^{{*}} = {z_star} $

$ X^{{*}} = \\begin{{bmatrix}}
    { '\n'.join([f'{x:.2f} \\\\' for x in X]) }
\\end{{bmatrix}}
$

$ I^{{*}} = $ {I_star.tolist()}

'''
display(
    Markdown(
        result_md
    )
)

# Exercicio 4.2

Verificando se as restrições de não negatividade para $ {A_{I}}^{-1} $ são violadas

In [ ]:
A = np.array([
    [ 2,  1, 3, -1,  0, 0],
    [-1,  1, 0,  0, -1, 0],
    [-1, -5, 1,  0,  0, 1]
])

B = np.array([
    3,
    1,
    4
])

C = np.array([1, 3, -1, 0, 0, 0])

I = [3, 4, 5]

A_I = A[:,I] # definição da matriz básica
A_I_inv = np.linalg.inv(A_I)

result = f'Restrições de negatividade{
    ' não ' if np.all(A_I_inv @ B >= 0) else ' '
}violadas'

display(
    Markdown(
        result
    )
)

Como as restrições de negatividade foram violadas, devemos criar um problema artificial e resolvê-lo para:

1. Verificar a factibilidade do problema original
2. Encontrar uma base inicial factível para o problema original que seja capaz de resolvê-lo

O problema artificial é do tipo:

$$
\begin{aligned}
    & \text{min} \quad \phi = & \textbf{1} \: x_a \\
    & \text{s. t.} & A x + x_a &= b \\
    &              &       x   &\geq 0 \\
    &              &       x_a &\geq 0 \\
\end{aligned}
$$

Vamos resolvê-lo abaixo:

In [ ]:

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = [6, 7, 5]

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)

Verificamos que $ \phi^{*} = 0 $, o que caracteriza o problema original como factível.

Portanto, vamos resolvê-lo à partir da base inicial facítvel obtida na resolução do artificial $ I = [2, 1, 5] $ já que nessa base não existem variáveis artificiais

In [ ]:
I = [2, 1, 5]

# resolvendo problema original com o simplex tableau revisado
solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)

## Result Comparison through scipy

In [ ]:
result = linprog(C, A_eq = A, b_eq = B, method='highs')
print('Result comparison, we found the optimal solution through our tableaus')
display(result)

## Result Comparison through Simplex Two Phases (without tableaus)

In [ ]:
# I = [3, 4, 5]
_, n = A.shape
X = np.zeros(n)
z_star, x_star, I_star, A_I_star, A, iterations_count, solution_type, debug_info  = simplex_two_phases(A, B, C, I, debug=True)
X[I_star] = x_star

result_md = f'''
Solution Type: {solution_types[solution_type]}

$ Z^{{*}} = {z_star} $

$ X^{{*}} = \\begin{{bmatrix}}
    { '\n'.join([f'{x:.2f} \\\\' for x in X]) }
\\end{{bmatrix}}
$

$ I^{{*}} = $ {I_star.tolist()}

'''
display(
    Markdown(
        result_md
    )
)

# Exercicio 4.3

Verificando se as restrições de não negatividade para $ {A_{I}}^{-1} $ são violadas

In [ ]:
A = np.array([
    [3,  1, -1, 1,  0,  0],
    [4, -1,  2, 0, -1,  0],
    [2,  3, -2, 0,  0, -1]
])

B = np.array([
    8,
    2,
    4
])

C = np.array([-2, 1, -1, 0, 0, 0])

I = [3, 4, 5]

A_I = A[:,I] # definição da matriz básica
A_I_inv = np.linalg.inv(A_I)

result = f'Restrições de negatividade{
    ' não ' if np.all(A_I_inv @ B >= 0) else ' '
}violadas'

display(
    Markdown(
        result
    )
)

Como as restrições de negatividade foram violadas, devemos criar um problema artificial e resolvê-lo para:

1. Verificar a factibilidade do problema original
2. Encontrar uma base inicial factível para o problema original que seja capaz de resolvê-lo

O problema artificial é do tipo:

$$
\begin{aligned}
    & \text{min} \quad \phi = & \textbf{1} \: x_a \\
    & \text{s. t.} & A x + x_a &= b \\
    &              &       x   &\geq 0 \\
    &              &       x_a &\geq 0 \\
\end{aligned}
$$

Vamos resolvê-lo abaixo:

In [ ]:

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = [3, 6, 7]

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)

Verificamos que $ \phi^{*} = 0 $, o que caracteriza o problema original como factível.

Portanto, vamos resolvê-lo à partir da base inicial facítvel obtida na resolução do artificial $ I = [3, 0, 1] $ já que nessa base não existem variáveis artificiais

In [ ]:
I = [3, 0, 1]

# resolvendo problema original com o simplex tableau revisado
solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)

## Result Comparison through scipy

In [ ]:
result = linprog(C, A_eq = A, b_eq = B, method='highs')
print('Result comparison, we found the optimal solution through our tableaus')
display(result)

## Result Comparison through Simplex Two Phases (without tableaus)

In [ ]:
I = [3, 4, 5]
_, n = A.shape
X = np.zeros(n)
z_star, x_star, I_star, A_I_star, A, iterations_count, solution_type, debug_info  = simplex_two_phases(A, B, C, I, debug=True)
X[I_star] = x_star

result_md = f'''
Solution Type: {solution_types[solution_type]}

$ Z^{{*}} = {z_star} $

$ X^{{*}} = \\begin{{bmatrix}}
    { '\n'.join([f'{x:.2f} \\\\' for x in X]) }
\\end{{bmatrix}}
$

$ I^{{*}} = $ {I_star.tolist()}

'''
display(
    Markdown(
        result_md
    )
)

# Exercicio 4.4

Verificando se as restrições de não negatividade para $ {A_{I}}^{-1} $ são violadas

In [ ]:
A = np.array([
    [1,  2, 1, 1,  0,  0, 0],
    [1, -1, 0, 0, -1,  0, 0],
    [1, -1, 0, 0,  0, -1, 0],
    [1,  3, 1, 0,  0,  0, 1]
])

B = np.array([
    10,
    10,
    6,
    14
])

C = np.array([-4, -5, 3, 0, 0, 0, 0])

I = list(range(3,7))

A_I = A[:,I] # definição da matriz básica
A_I_inv = np.linalg.inv(A_I)

result = f'Restrições de negatividade{
    ' não ' if np.all(A_I_inv @ B >= 0) else ' '
}violadas'

display(
    Markdown(
        result
    )
)

Como as restrições de negatividade foram violadas, devemos criar um problema artificial e resolvê-lo para:

1. Verificar a factibilidade do problema original
2. Encontrar uma base inicial factível para o problema original que seja capaz de resolvê-lo

O problema artificial é do tipo:

$$
\begin{aligned}
    & \text{min} \quad \phi = & \textbf{1} \: x_a \\
    & \text{s. t.} & A x + x_a &= b \\
    &              &       x   &\geq 0 \\
    &              &       x_a &\geq 0 \\
\end{aligned}
$$

Vamos resolvê-lo abaixo:

In [ ]:

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = [3, 7, 8, 6]

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)

Verificamos que $ \phi^{*} = 0 $, o que caracteriza o problema original como factível.

Vemos ainda que uma variável artificial está na base — $ x_7 $ — o que implica a necessidade de sua troca (ou remoção, se impossível, a partir da retirada de sua restrição equivalente) para resolver o problema original.

Vamos avaliar o conjunto $J$ das variáveis não básicas originais onde $ J = [1, 2, 3, 4] $ e seu respectivo $ \hat{C_{J}} = [-3, -1, -1, -1] $

Apesar de todo $\hat{C_{J}}$ ser negativo, podemos inserir uma variável não básica original que piore a solução do problema desde que o conjunto base $I$ resultante mantenha as restrições de factibilidade adequadas.

Pelo critério de entrada, do indice mais à esquerda (critério de bland) do maior valor de $\hat{C_{J}}$, $k = 2$

Logo, vamos tentar substituir $x_7$ artificial por $x_2$ não básica original.

Critérios:

1. $ {A_{\text{artificial}}}_{[7,k]} \neq 0 $

Lembrando que a linha da matriz de restrições correspondente à variável básica $x_7$ neste momento é a linha 2

In [ ]:
md = f'''
$x_2$ { 'pode' if A_artificial[1,2] else 'não pode '} entrar na base.

$ {{A_{{\\text{{artificial}}}}}}_{{[7,2]}} = {A_artificial[1,2]}
$ '''
display(Markdown(md))

Vamos testar agora com o próximo valor de $\hat{C_J}$ que corresponde ao 3º índice, $k = 3$

In [ ]:
md = f'''
$x_3$ { 'pode' if A_artificial[1,3] else 'não pode '} entrar na base.

$ {{A_{{\\text{{artificial}}}}}}_{{[7,3]}} = {A_artificial[1,3]}
$ '''
display(Markdown(md))

Vamos testar agora com o último valor possível de $\hat{C_J}$ que corresponde ao 4º índice, $k = 4$

In [ ]:
md = f'''
$x_4$ { 'pode' if A_artificial[1,4] else 'não pode '} entrar na base.

$ {{A_{{\\text{{artificial}}}}}}_{{[7,4]}} = {A_artificial[1,4]}
$ '''
display(Markdown(md))

Ainda existem mais critérios que devem ser levados em consideração:

2. Se a matriz $A_I$ resultante é não-singular (invertível)

In [ ]:
I = [5, 4, 0, 6]
A_I = A_artificial[:,I]

md = f'''
$A_I$ resultante é {
    'singular.' if np.linalg.det(A_I) == 0 else 'não singular.'
}
'''
display(Markdown(md))

3. Por fim, se os critérios de não negatividade — $ X_I \geq 0 $ — são respeitados, onde:

$$
    X_I = {A_I}^{-1} B
$$

In [ ]:
md = f'''
Os critérios de negatividade{
    ' não ' if np.any(np.linalg.inv(A_I).dot(B) < 0) else ' '
}foram respeitados
'''
display(Markdown(md))

Então, a substituição é de $x_7$ por $x_4$ é válida e vamos utilizar $ I = [5, 4, 0, 6] $ como base inicial factível para o simplex tableau revisto de forma a solucionar o problema original:

In [ ]:
I = [5, 4, 0, 6]

# resolvendo problema original com o simplex tableau revisado
solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)

## Result Comparison through scipy

In [ ]:
result = linprog(C, A_eq = A, b_eq = B, method='highs')
print('Result comparison, we found the optimal solution through our tableaus')
display(result)

## Result Comparison through Simplex Two Phases (without tableaus)

In [ ]:
I = [3, 4, 5, 6]
_, n = A.shape
X = np.zeros(n)
z_star, x_star, I_star, A_I_star, A, iterations_count, solution_type, debug_info  = simplex_two_phases(A, B, C, I, debug=True)
X[I_star] = x_star

result_md = f'''
Solution Type: {solution_types[solution_type]}

$ Z^{{*}} = {z_star} $

$ X^{{*}} = \\begin{{bmatrix}}
    { '\n'.join([f'{x:.2f} \\\\' for x in X]) }
\\end{{bmatrix}}
$

$ I^{{*}} = $ {I_star}

'''
display(
    Markdown(
        result_md
    )
)